In [1]:
import os
import random
import json
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Lista de archivos a procesar
DATASETS = [
    "Amazon_Stock_Price_output.csv",
    "Euro_Stoxx_50_Stock_Price_output.csv",
    "Google_Stock_Price_output.csv",
    "Hang_Seng_Stock_Price_output.csv",
    "IBEX_35_Stock_Price_output.csv",
    "Indra_Stock_Price_output.csv",
    "P&G_Stock_Price_output.csv",
    "S&P500_Stock_Price_output.csv"
]

BASE_PATH = "Conjunt de dades Preprocessades/Datasets"  # Carpeta donde están los CSV

# Parámetros generales
N_STEPS = 30        # longitud de secuencia
TEST_RATIO = 0.10   # 10% para test
VAL_RATIO = 0.10    # 10% para validación
TRAIN_RATIO = 1 - TEST_RATIO - VAL_RATIO

N_TRIALS = 20       # número de combinaciones aleatorias de hiperparámetros
EPOCHS = 50         # épocas para búsqueda rápida
BATCH_SIZE = 32

FEATURE_COLUMNS = [
    "Open", "High", "Low", "Volume",
    "EMA_7", "EMA_40", "MACD", "Signal_Line",
    "MACD_Hist", "RSI", "ATR"
]
TARGET_COLUMN = "Close"


def create_sequences(X, y, n_steps=30):
    Xs, ys = [], []
    for i in range(n_steps, len(X)):
        Xs.append(X[i - n_steps:i])
        ys.append(y[i])
    return np.array(Xs), np.array(ys)


def build_lstm_model(sequence_length, n_features, units, n_layers, dropout, learning_rate):
    """
    Construye y compila un modelo LSTM con los parámetros dados.
    """
    model = Sequential()
    for i in range(n_layers):
        is_first = (i == 0)
        is_last = (i == n_layers - 1)
        return_sequences = not is_last
        if is_first:
            model.add(LSTM(
                units,
                return_sequences=return_sequences,
                input_shape=(sequence_length, n_features)
            ))
        else:
            model.add(LSTM(
                units,
                return_sequences=return_sequences
            ))
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="huber", optimizer=optimizer, metrics=["mean_absolute_error"])
    return model


def compute_metrics(model, X_scaled, y_scaled, scaler_y):
    """
    Calcula MAE, RMSE y R2 entre y_scaled y las predicciones inversamente escaladas de model.predict(X_scaled).
    Devuelve una tupla (mae, rmse, r2).
    """
    y_pred_scaled = model.predict(X_scaled, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_scaled)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2


def process_dataset(filename):
    """
    Ejecuta todo el pipeline sobre un único dataset:
    - Carga y preprocesa
    - División train/val/test
    - Escalado
    - Búsqueda aleatoria de hiperparámetros
    - Entrenamiento final del mejor modelo
    - Cálculo y guardado de métricas test
    - Guardado de pesos y parámetros del mejor modelo
    """
    dataset_name = os.path.splitext(filename)[0]  # e.g. "Amazon_Stock_Price_output"
    print(f"\n=== Procesando dataset: {dataset_name} ===")

    # 1) Cargar CSV
    path = os.path.join(BASE_PATH, filename)
    df = pd.read_csv(path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.sort_values("Date", inplace=True)

    # 2) Eliminar filas con NaN en features+target
    df.dropna(subset=FEATURE_COLUMNS + [TARGET_COLUMN], inplace=True)

    # 3) Crear secuencias
    data_X = df[FEATURE_COLUMNS].values
    data_y = df[TARGET_COLUMN].values.reshape(-1, 1)
    X_seq, y_seq = create_sequences(data_X, data_y, n_steps=N_STEPS)

    # 4) División temporal train/val/test
    n_total = len(X_seq)
    train_end = int(n_total * TRAIN_RATIO)
    val_end = train_end + int(n_total * VAL_RATIO)

    X_train = X_seq[:train_end]
    y_train = y_seq[:train_end]

    X_val = X_seq[train_end:val_end]
    y_val = y_seq[train_end:val_end]

    X_test = X_seq[val_end:]
    y_test = y_seq[val_end:]

    # 5) Escalado
    #    - Fit scaler_X solo en X_train
    #    - Transform en train, val, test
    scaler_X = MinMaxScaler()
    X_train_flat = X_train.reshape(-1, len(FEATURE_COLUMNS))
    X_train_scaled_flat = scaler_X.fit_transform(X_train_flat)
    X_train_scaled = X_train_scaled_flat.reshape(X_train.shape)

    # Val
    X_val_flat = X_val.reshape(-1, len(FEATURE_COLUMNS))
    X_val_scaled_flat = scaler_X.transform(X_val_flat)
    X_val_scaled = X_val_scaled_flat.reshape(X_val.shape)

    # Test
    X_test_flat = X_test.reshape(-1, len(FEATURE_COLUMNS))
    X_test_scaled_flat = scaler_X.transform(X_test_flat)
    X_test_scaled = X_test_scaled_flat.reshape(X_test.shape)

    # 6) Escalar target con scaler_y
    scaler_y = MinMaxScaler()
    y_train_scaled = scaler_y.fit_transform(y_train)
    y_val_scaled = scaler_y.transform(y_val)
    y_test_scaled = scaler_y.transform(y_test)

    # 7) Búsqueda aleatoria de hiperparámetros en Training+Validation
    best_val_loss = np.inf
    best_params = None
    best_model = None

    for trial in range(N_TRIALS):
        # Seleccionar aleatorio de cada lista
        n_layers = random.choice([1, 2, 3])
        units = random.choice([64, 128, 256, 512])
        dropout = random.choice([0.1, 0.3, 0.5])
        lr = random.choice([1e-5, 1e-4, 1e-3])

        model = build_lstm_model(
            sequence_length=N_STEPS,
            n_features=len(FEATURE_COLUMNS),
            units=units,
            n_layers=n_layers,
            dropout=dropout,
            learning_rate=lr
        )

        callbacks = [
            EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True, verbose=0),
            ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=0, min_lr=1e-6)
        ]

        history = model.fit(
            X_train_scaled, y_train_scaled,
            validation_data=(X_val_scaled, y_val_scaled),
            epochs=EPOCHS,
            batch_size=BATCH_SIZE,
            callbacks=callbacks,
            verbose=0
        )

        current_val_loss = min(history.history["val_loss"])
        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            best_params = (n_layers, units, dropout, lr)
            best_model = model

        print(f" Trial {trial+1}/{N_TRIALS} - val_loss: {current_val_loss:.6f}  "
              f"(layers={n_layers}, units={units}, dropout={dropout}, lr={lr})")

    print(f"→ Mejor val_loss: {best_val_loss:.6f} con params: "
          f"layers={best_params[0]}, units={best_params[1]}, "
          f"dropout={best_params[2]}, lr={best_params[3]}")

    # 8.1) Guardar hiperparámetros en JSON
    params_folder = os.path.join("resultats_LSTM", dataset_name)
    os.makedirs(params_folder, exist_ok=True)
    params_path = os.path.join(params_folder, f"{dataset_name}_best_params.json")

    with open(params_path, "w") as fp:
        json.dump({
            "n_layers":      best_params[0],
            "units":         best_params[1],
            "dropout":       best_params[2],
            "learning_rate": best_params[3]
        }, fp)

    print(f" Hiperparámetros guardados en: {params_path}")

    # 8) Evaluar mejor modelo en Test
    mae_test, rmse_test, r2_test = compute_metrics(best_model, X_test_scaled, y_test_scaled, scaler_y)

    # 9) Guardar métricas Test a CSV
    df_metrics = pd.DataFrame({
        "Dataset": [dataset_name],
        "MAE":     [mae_test],
        "RMSE":    [rmse_test],
        "R2":      [r2_test]
    })
    metrics_folder = os.path.join("resultats_LSTM", dataset_name)
    os.makedirs(metrics_folder, exist_ok=True)
    metrics_path = os.path.join(metrics_folder, f"{dataset_name}_metrics_test.csv")
    df_metrics.to_csv(metrics_path, index=False)
    print(f" Métricas guardadas en: {metrics_path}")

    # 10) Guardar pesos del mejor modelo
    weights_path = os.path.join(metrics_folder, f"{dataset_name}_best_weights.weights.h5")
    best_model.save_weights(weights_path)
    print(f" Pesos guardados en: {weights_path}")


if __name__ == "__main__":
    # Crear carpeta principal de resultats
    os.makedirs("resultats_LSTM", exist_ok=True)

    for fname in DATASETS:
        process_dataset(fname)



=== Procesando dataset: Amazon_Stock_Price_output ===


c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 Trial 1/20 - val_loss: 0.002221  (layers=2, units=256, dropout=0.5, lr=1e-05)
 Trial 2/20 - val_loss: 0.001122  (layers=2, units=256, dropout=0.1, lr=0.0001)
 Trial 3/20 - val_loss: 0.016056  (layers=1, units=64, dropout=0.5, lr=1e-05)
 Trial 4/20 - val_loss: 0.000947  (layers=3, units=128, dropout=0.5, lr=0.001)
 Trial 5/20 - val_loss: 0.001289  (layers=1, units=256, dropout=0.3, lr=1e-05)
 Trial 6/20 - val_loss: 0.000454  (layers=1, units=512, dropout=0.1, lr=0.001)
 Trial 7/20 - val_loss: 0.001403  (layers=2, units=64, dropout=0.5, lr=0.001)
 Trial 8/20 - val_loss: 0.004922  (layers=2, units=64, dropout=0.1, lr=1e-05)
 Trial 9/20 - val_loss: 0.001414  (layers=2, units=512, dropout=0.1, lr=1e-05)
 Trial 10/20 - val_loss: 0.001382  (layers=3, units=64, dropout=0.3, lr=0.0001)
 Trial 11/20 - val_loss: 0.005753  (layers=2, units=64, dropout=0.1, lr=1e-05)
 Trial 12/20 - val_loss: 0.001435  (layers=2, units=64, dropout=0.5, lr=0.0001)
 Trial 13/20 - val_loss: 0.000606  (layers=1, units=

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 Trial 1/20 - val_loss: 0.000267  (layers=1, units=512, dropout=0.1, lr=0.0001)
 Trial 2/20 - val_loss: 0.006269  (layers=3, units=64, dropout=0.5, lr=1e-05)
 Trial 3/20 - val_loss: 0.004667  (layers=2, units=64, dropout=0.5, lr=0.0001)
 Trial 4/20 - val_loss: 0.000360  (layers=3, units=64, dropout=0.1, lr=0.0001)
 Trial 5/20 - val_loss: 0.001136  (layers=1, units=256, dropout=0.3, lr=0.0001)
 Trial 6/20 - val_loss: 0.000278  (layers=1, units=512, dropout=0.5, lr=0.001)
 Trial 7/20 - val_loss: 0.000405  (layers=3, units=128, dropout=0.1, lr=0.0001)
 Trial 8/20 - val_loss: 0.000459  (layers=3, units=512, dropout=0.5, lr=0.0001)
 Trial 9/20 - val_loss: 0.001376  (layers=3, units=64, dropout=0.5, lr=0.0001)
 Trial 10/20 - val_loss: 0.000570  (layers=2, units=64, dropout=0.3, lr=0.0001)
 Trial 11/20 - val_loss: 0.000579  (layers=1, units=512, dropout=0.3, lr=1e-05)
 Trial 12/20 - val_loss: 0.001231  (layers=3, units=256, dropout=0.3, lr=1e-05)
 Trial 13/20 - val_loss: 0.000203  (layers=1, 

...

 Trial 1/20 - val_loss: 0.001103  (layers=3, units=128, dropout=0.1, lr=0.001)
 Trial 2/20 - val_loss: 0.002013  (layers=2, units=256, dropout=0.5, lr=0.0001)
 Trial 3/20 - val_loss: 0.001402  (layers=3, units=512, dropout=0.1, lr=0.0001)
 Trial 4/20 - val_loss: 0.002421  (layers=3, units=512, dropout=0.3, lr=1e-05)
 Trial 5/20 - val_loss: 0.000905  (layers=1, units=512, dropout=0.5, lr=0.001)
 Trial 6/20 - val_loss: 0.006336  (layers=3, units=64, dropout=0.1, lr=0.0001)
 Trial 7/20 - val_loss: 0.001598  (layers=2, units=256, dropout=0.1, lr=0.0001)
 Trial 8/20 - val_loss: 0.002482  (layers=2, units=64, dropout=0.1, lr=0.001)
 Trial 9/20 - val_loss: 0.001802  (layers=2, units=256, dropout=0.1, lr=0.001)
 Trial 10/20 - val_loss: 0.001801  (layers=3, units=512, dropout=0.1, lr=0.001)
 Trial 11/20 - val_loss: 0.000977  (layers=2, units=256, dropout=0.3, lr=0.001)
 Trial 12/20 - val_loss: 0.000930  (layers=1, units=256, dropout=0.3, lr=0.0001)
 Trial 13/20 - val_loss: 0.002322  (layers=1, 

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 Trial 1/20 - val_loss: 0.000618  (layers=1, units=128, dropout=0.5, lr=0.001)
 Trial 2/20 - val_loss: 0.003561  (layers=3, units=256, dropout=0.3, lr=1e-05)
 Trial 3/20 - val_loss: 0.004473  (layers=3, units=128, dropout=0.5, lr=1e-05)
 Trial 4/20 - val_loss: 0.000463  (layers=1, units=128, dropout=0.3, lr=0.001)
 Trial 5/20 - val_loss: 0.001104  (layers=3, units=512, dropout=0.1, lr=0.0001)
 Trial 6/20 - val_loss: 0.001167  (layers=2, units=256, dropout=0.1, lr=0.0001)
 Trial 7/20 - val_loss: 0.000649  (layers=2, units=128, dropout=0.3, lr=0.001)
 Trial 8/20 - val_loss: 0.000651  (layers=2, units=64, dropout=0.3, lr=0.001)
 Trial 9/20 - val_loss: 0.000677  (layers=3, units=256, dropout=0.1, lr=0.001)
 Trial 10/20 - val_loss: 0.000759  (layers=3, units=128, dropout=0.1, lr=0.001)
 Trial 11/20 - val_loss: 0.003510  (layers=1, units=128, dropout=0.5, lr=0.0001)
 Trial 12/20 - val_loss: 0.000757  (layers=2, units=128, dropout=0.3, lr=0.0001)
 Trial 13/20 - val_loss: 0.006462  (layers=2, 

c:\Users\jesus\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


 Trial 1/20 - val_loss: 0.001440  (layers=1, units=128, dropout=0.1, lr=0.0001)
 Trial 2/20 - val_loss: 0.003107  (layers=3, units=64, dropout=0.5, lr=0.0001)
 Trial 3/20 - val_loss: 0.000633  (layers=1, units=128, dropout=0.1, lr=0.0001)
 Trial 4/20 - val_loss: 0.000409  (layers=1, units=64, dropout=0.1, lr=0.0001)
 Trial 5/20 - val_loss: 0.000722  (layers=3, units=128, dropout=0.5, lr=0.0001)
 Trial 6/20 - val_loss: 0.000457  (layers=1, units=256, dropout=0.3, lr=0.001)
 Trial 7/20 - val_loss: 0.001985  (layers=3, units=64, dropout=0.3, lr=0.001)
 Trial 8/20 - val_loss: 0.000395  (layers=2, units=256, dropout=0.5, lr=0.001)
 Trial 9/20 - val_loss: 0.008657  (layers=1, units=64, dropout=0.5, lr=1e-05)
 Trial 10/20 - val_loss: 0.001508  (layers=3, units=256, dropout=0.3, lr=1e-05)
 Trial 11/20 - val_loss: 0.001956  (layers=3, units=256, dropout=0.3, lr=1e-05)
 Trial 12/20 - val_loss: 0.000219  (layers=2, units=128, dropout=0.1, lr=0.001)
 Trial 13/20 - val_loss: 0.000926  (layers=1, un